In [302]:
# Loading all the libraries needed for running the code chunks below

import json
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException


In [289]:
# Functions

def parse(main_link):
    
    driver.get(main_link)
    driver.implicitly_wait(1)
    html = driver.page_source
    data = []
    titles = []
    links, next_page_link = parse_page_results(main_link)
    
    for i in range(0,19):                                                 
        new_links, next_page_link = parse_page_results(next_page_link)
        links = np.concatenate((links, new_links), axis=0)
    
    # movie data parsing
    for i in range(0,len(links)):
        print(i)
        movie_data, new_titles = parse_movie(links[i])
        data.append(movie_data)
        titles.extend(new_titles)
    print(len(titles))
    
    return data, titles



def parse_page_results(link):
    
    driver.get(link)
    driver.implicitly_wait(1)
    html = driver.page_source
    results = driver.find_elements(By.CLASS_NAME, 'clamp-summary-wrap')
    links = np.empty(len(results), dtype='U150')
    for i in range(len(results)):
        a = results[i].find_elements(By.TAG_NAME, 'a')
        href = a[1].get_attribute('href')
        if href:
            links[i] = href
    
    nxt = driver.find_element(By.CLASS_NAME, 'next') 
    next_page_link = nxt.find_element(By.TAG_NAME, 'a').get_attribute('href')
    
    return links, next_page_link



def parse_movie(link):   
    
    movie_data = {'title':'','year':'', 'genre':'', 'summary':'', 'starring':'', 'director':'', 'runtime':''}
    driver.get(link)
    driver.implicitly_wait(1)
    html = driver.page_source
    titles = []
    
    title = driver.find_element(By.TAG_NAME, 'h1')
    movie_data['title'] = title.text
    titles.append(title.text)
    
    try:
        year = driver.find_element(By.CLASS_NAME, 'release_year')
        movie_data['year'] = year.text
    except NoSuchElementException:
        pass 
     
    try:
        genres = driver.find_element(By.CLASS_NAME, 'genres')
        genres = genres.find_elements(By.TAG_NAME, 'span')
        genres_list = []
        for i in range(2,len(genres)):
            genres_list.append(genres[i].text)
        movie_data['genre'] = genres_list
    except NoSuchElementException:
        pass 

    summary = driver.find_element(By.CLASS_NAME, 'summary_deck')

    try:
        button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
        button.click()
    except NoSuchElementException:
        pass
   
    try:
        expand_button = summary.find_element(By.CLASS_NAME, 'toggle_expand_collapse')
        expand_button.click()
        summary = summary.find_elements(By.CLASS_NAME, 'blurb')
        movie_data['summary'] = summary[1].text
    except ElementClickInterceptedException:
        summary = driver.find_element(By.CLASS_NAME, 'summary_deck')
        summary = summary.find_elements(By.TAG_NAME, 'span')
        movie_data['summary'] = summary[1].text
    except NoSuchElementException:
        summary = driver.find_element(By.CLASS_NAME, 'summary_deck')
        summary = summary.find_elements(By.TAG_NAME, 'span')
        movie_data['summary'] = summary[1].text

    try:
        cast = driver.find_element(By.CLASS_NAME, 'summary_cast')
        cast = cast.find_elements(By.TAG_NAME, 'a')
        cast_list = []
        for i in range(len(cast)):
            cast_list.append(cast[i].text)
        movie_data['starring'] = cast_list
    except NoSuchElementException:
        pass    
    
    try:
        director = driver.find_element(By.CLASS_NAME, 'director')
        director = director.find_elements(By.TAG_NAME, 'span')
        movie_data['director'] = director[1].text
    except NoSuchElementException:
        pass
    
    try:
        runtime = driver.find_element(By.CLASS_NAME, 'runtime')
        runtime = runtime.find_elements(By.TAG_NAME, 'span')
        movie_data['runtime'] = runtime[1].text
    except NoSuchElementException:
        pass
    
    return movie_data, titles

In [290]:
# Main code chunk

driver = webdriver.Chrome('chromedriver')
main_link = 'https://www.metacritic.com/browse/movies/score/userscore/all/filtered?view=detailed'
start = time.time()
data, titles = parse(main_link)
end = time.time()
print('Parsing time:', end-start)
driver.close()

with open("metacritic_data1.json", "w", encoding='utf-8') as jsonfile:
    json.dump(data, jsonfile, ensure_ascii=False, indent = 2)
    
with open('titles.txt', 'w') as f:
    f.write(str(titles))

C:\Users\Asus\AppData\Local\Temp\ipykernel_11000\2160487578.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Asus/Desktop/FAKS/NLP/chromedriver')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
Parsing time: 10783.404063940048


In [301]:
with open("titles.csv", "w", newline="", encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(titles)

['The Godfather', 'The Lord of the Rings: The Fellowship of the Ring', 'Memento', 'Pulp Fiction', 'Star Wars: Episode V - The Empire Strikes Back', 'The Godfather: Part II', 'Joker', 'Whiplash', 'Toy Story 3', 'The Dark Knight', 'The Lord of the Rings: The Return of the King', 'The Lord of the Rings: The Two Towers', 'Spirited Away', 'Fight Club', 'The Matrix', 'The Shawshank Redemption', 'Terminator 2: Judgment Day', 'Aliens', 'Back to the Future', 'Raiders of the Lost Ark', 'Puss in Boots: The Last Wish', 'Promare', 'Your Name.', 'The Dark Knight Rises', 'A Separation', 'Inception', 'WALL-E', "Howl's Moving Castle", 'The Best of Youth', 'House of Flying Daggers', 'The Incredibles', 'Hero', 'Eternal Sunshine of the Spotless Mind', 'Pirates of the Caribbean: The Curse of the Black Pearl', 'City of God', 'Shrek', 'Traffic', 'Gladiator', 'Princess Mononoke', 'American Beauty', 'Saving Private Ryan', 'Out of Sight', 'Face/Off', 'Selena', 'Toy Story', 'Seven', 'The Lion King', "Schindler's